In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_pickle("/kaggle/input/data0-25/data_norm0.25.pkl")
df2 = pd.read_pickle("/kaggle/input/data1-0/data_norm1.pkl")
df3 = pd.read_pickle("/kaggle/input/data0-5/data_norm0.5.pkl")

In [3]:
df = pd.concat([df1, df2])
df = pd.concat([df, df3])

In [4]:
df = df.reset_index() 

In [6]:
df = df.drop("index",  axis=1)

In [7]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.data.iloc[idx, 0], dtype=torch.float)
        y = torch.tensor(self.data.iloc[idx, 1], dtype=torch.float)
        return x, y

dataset = CustomDataset(df) #!!!!!!! df_train
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [8]:
import torch
import torch.nn as nn


class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(989, 989)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(989, 989)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(989, 256)
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(256, 49)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

model = CustomModel()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

Найдём нужный threshold бинпоиском

In [50]:
import numpy
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

threshold = 1
left = 0
right = 1
counter = 0
last_recall = 0


while precision < 1:
    model = torch.load("/kaggle/input/version3/pytorch/models/1/989_x2_256_49_epoch2.pth")
    
    precision_list = []
    recall_list = []
    
    
    for targets, inputs in dataloader:
        out = model.forward(inputs).detach().numpy()
        targets = targets.detach().numpy()
        out = np.argmax(out, axis=1)
        targets = np.argmax(targets, axis=1)
        out = (out >= threshold).astype(int)
        targets = (targets >= threshold).astype(int)


        precision = precision_score(targets, out, average=None, zero_division=1)
        recall = recall_score(targets, out, average=None, zero_division=1)

        precision_list.extend(precision)
        recall_list.extend(recall)
        
        if counter == 100:
            val = sum(recall_list)/len(recall_list)
            pre = sum(precision_list)/len(precision_list)
            if  val > last_recall:
                left = threshold
                threshold += (left+right)/2
            else:
                right = threshold
                threshold -= (left+right)/2
                
            print(round(val, 3), round(pre, 3), round(threshold, 3))
            precision_list = []
            recall_list = []
            counter = 0
            last_recall = val
            if 0.48 <= pre <= 0.52:
                print("susces",threshold, val, sum(precision_list)/len(precision_list))
                break

        counter += 1
            
        

0.9503990255376343 0.9181577620967742 2.0
0.9545423117367812 0.9511429742244035 3.5
0.9361767937572116 0.947784935945711 0.75
0.9569342619745846 0.898992808267002 2.875
0.929313117418825 0.9639526481418951 1.0625
0.9501219469895603 0.9604683941151545 3.03125
0.9332040273566015 0.9390060952406704 0.984375
0.9605176871447162 0.889444644202452 2.9921875
0.9179202130911513 0.9490539601900227 1.00390625
0.957772386364666 0.9714063795394801 3.001953125
0.9238725913412779 0.9176592637737836 0.9990234375
0.9600656604184831 0.924388350043592 2.99951171875
0.9197600387172209 0.9438568044523834 1.000244140625
0.9745314619033713 0.948365696521896 3.0001220703125
0.9172708162400506 0.9362565261882603 0.99993896484375
0.9733573805866433 0.9413196730305026 2.999969482421875
0.9049912583408134 0.9433147226361911 1.0000152587890625
0.958568102735233 0.9597756028269098 3.0000076293945312
0.9266316345443678 0.9195883682860924 0.9999961853027344
0.9523350018539117 0.9329602335928809 2.999998092651367
0.90

KeyboardInterrupt: 